#### Ilmatieteen laitoksen rajapinnan käyttö Pythonia ja Jupyter labiä käyttäen

https://en.ilmatieteenlaitos.fi/open-data-manual-accessing-data

In [1]:
# Importataan tarvittavat kirjastot
import datetime
import requests
import xmltodict
import json

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

from owslib.wfs import WebFeatureService

##### Tutkitaan ensin WFS rajapinnan ominaisuuksia

In [2]:
base_url = "https://opendata.fmi.fi/wfs" # URL, johon rajapinta vastaa.

In [3]:
wfs = WebFeatureService(url=base_url, version="2.0.0") # Dokumentaatiosta selvitetään, että rajapinta käyttää WFS versio 2.0.0.

In [5]:
wfs.identification.title # Tarkistetaan, että ollaan oikeassa rajapinnassa.

"Finnish Meteorological Institute's Open Data Download Service"

In [4]:
list(wfs.contents) # Saatavilla olevat WFS-polut.

['BsWfs:BsWfsElement',
 'avi:VerifiableMessage',
 'ef:EnvironmentalMonitoringFacility',
 'ef:EnvironmentalMonitoringNetwork',
 'omso:GridSeriesObservation',
 'omso:PointObservation',
 'omso:PointTimeSeriesObservation',
 'omso:ProfileObservation',
 'omso:TrajectoryObservation']

In [6]:
[operation.name for operation in wfs.operations] # Kaikki operaatiot, joita rajapinnassa voi tehdä. GetFeature on meidän tarkoitukseen sopiva, sillä sen avulla tiedon voi ladata.

['GetCapabilities',
 'DescribeFeatureType',
 'ListStoredQueries',
 'DescribeStoredQueries',
 'GetPropertyValue',
 'GetFeature',
 'version',
 'ImplementsBasicWFS',
 'ImplementsTransactionalWFS',
 'ImplementsLockingWFS',
 'KVPEncoding',
 'XMLEncoding',
 'SOAPEncoding',
 'ImplementsInheritance',
 'ImplementsRemoteResolve',
 'ImplementsResultPaging',
 'ImplementsStandardJoins',
 'ImplementsSpatialJoins',
 'ImplementsTemporalJoins',
 'ImplementsFeatureVersioning',
 'ManageStoredQueries',
 'CountDefault',
 'QueryExpressions']

##### Haetaan kaikki mahdolliset kyselyt, mitä palvelimelle voi tehdä
WFS versio 2.0.0. käyttää "storedqueries" ominaisuutta, jolla voi hakea suodattamalla tietoa rajapinnasta

In [7]:
storedqueries = []
for i, storedquery in enumerate(wfs.storedqueries):
    result = {}
    result["key"] = i
    result["id"] = storedquery.id
    storedqueries.append(result)

storedqueries

[{'key': 0, 'id': 'fmi::avi::observations::finland::iwxxm'},
 {'key': 1, 'id': 'fmi::avi::observations::finland::latest::iwxxm'},
 {'key': 2, 'id': 'fmi::avi::observations::iwxxm'},
 {'key': 3, 'id': 'fmi::avi::observations::latest::iwxxm'},
 {'key': 4, 'id': 'fmi::ef::networks'},
 {'key': 5, 'id': 'fmi::ef::stations'},
 {'key': 6, 'id': 'fmi::forecast::climatology::scenario::grid'},
 {'key': 7,
  'id': 'fmi::forecast::enfuser::airquality::helsinki-metropolitan::grid'},
 {'key': 8, 'id': 'fmi::forecast::harmonie::hybrid::grid'},
 {'key': 9,
  'id': 'fmi::forecast::harmonie::hybrid::point::multipointcoverage'},
 {'key': 10, 'id': 'fmi::forecast::harmonie::hybrid::point::simple'},
 {'key': 11, 'id': 'fmi::forecast::harmonie::hybrid::point::timevaluepair'},
 {'key': 12, 'id': 'fmi::forecast::harmonie::pressure::grid'},
 {'key': 13,
  'id': 'fmi::forecast::harmonie::pressure::point::multipointcoverage'},
 {'key': 14, 'id': 'fmi::forecast::harmonie::pressure::point::simple'},
 {'key': 15, '

Kysely, joka vastaa avain 113 on meidän tarkoitukseemme sopiva. Dokumentaatiosta selviää, että se pitää sisällään kuluneet säätilat alueittain.

In [32]:
# Tarkistetaan mahdolliset parametrit halutusta kyselystä.

sq_key = 113 # Key of the stored query in the stored queries response.

[parameter.name for parameter in wfs.storedqueries[sq_key].parameters]

['starttime',
 'endtime',
 'timestep',
 'parameters',
 'crs',
 'bbox',
 'place',
 'fmisid',
 'maxlocations',
 'geoid',
 'wmo',
 'timezone']

##### Ladataan ja parsetaan data

In [15]:
# Testataan ensin pienellä kyselyllä.
# Haetaan ensimmäisen 7 päivän säät vuoden 2020 alusta.
# Kyseisen kyselyn maksimi aikaväli on 7 päivää kerralla.

start_time = datetime.datetime(2020,1,1, 0, 0 , 0)
end_time = datetime.datetime(2020,1,7, 0, 0, 0)

query = "fmi::observations::weather::timevaluepair" # Kyselytyyppi rajapinnalle

parameters = {
    "service": "WFS", # WFS -palvelu
    "version": "2.0.0", # WFS versio
    "request": "GetFeature", # WFS ominaisuus. GetFeature hakee ja lataa dataa.
    "storedquery_id": query, # Kysely ID.
    "starttime": start_time, # datetime objekti.
    "endtime": end_time, # datetime objekti
    "place": "Sastamala", # Paikan nimi. Hakee automaattisesti lähimmän sääaseman.
    "maxlocations": 1, # Kuinka monelta asemalta tietoa haetaan?
    #"parameters":"t2m" # Mahdolliset ilmatieteelliset parametrit. Jos jätetään tyhjäksi, hakee kaikki.
}

In [16]:
# Rakennetaan kysely URL. 

base_url = "http://opendata.fmi.fi/wfs/fin?"
url = base_url

for param in parameters:
    value = parameters[param]
    if isinstance(value, datetime.date):
        value = value.strftime("%Y-%m-%dT%H:%M:%SZ")
    url += param + "=" +str(value)+"&"
    
url

'http://opendata.fmi.fi/wfs/fin?service=WFS&version=2.0.0&request=GetFeature&storedquery_id=fmi::observations::weather::timevaluepair&starttime=2020-01-01T00:00:00Z&endtime=2020-01-07T12:00:00Z&place=Sastamala&maxlocations=1&timestep=720&'

In [17]:
# Lähetään kysely ja odotetaan vastausta.
response = requests.get(url)

In [13]:
# Tarvitaan vain vastauksen teksti. Teksti on XML-muodossa, joten parsetaan se kirjastolla python kirjastoksi.
data = xmltodict.parse(response.text) 

In [264]:
# Tarkistetaan, miltä näyttää.
data

OrderedDict([('wfs:FeatureCollection',
              OrderedDict([('@timeStamp', '2022-01-25T15:07:47Z'),
                           ('@numberMatched', '13'),
                           ('@numberReturned', '13'),
                           ('@xmlns:wfs', 'http://www.opengis.net/wfs/2.0'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('@xmlns:xlink', 'http://www.w3.org/1999/xlink'),
                           ('@xmlns:om', 'http://www.opengis.net/om/2.0'),
                           ('@xmlns:ompr',
                            'http://inspire.ec.europa.eu/schemas/ompr/3.0'),
                           ('@xmlns:omso',
                            'http://inspire.ec.europa.eu/schemas/omso/3.0'),
                           ('@xmlns:gml', 'http://www.opengis.net/gml/3.2'),
                           ('@xmlns:gmd', 'http://www.isotc211.org/2005/gmd'),
                           ('@xmlns:gco', 'htt

Näin kysely suoritetaan rajapinnalle. Alla on vielä selitykset jokaiselle vastausparametrille. Selitykset ovat löytyneet rajapinnan kyselyvastauksesta.

#### Arvot ja niiden selitykset

lähde: vastauksien selitteet

t2m = lämpötila (\*C) <br>
ws_10min = tuulen nopeus (m/s) <br>
wg_10min = puuskatuuli (m/s) <br>
wd_10mmin = tuulen suunta (deg) <br>
rh = suhteellinen kosteus (%) <br>
td = kastepistelämpötila (\*C) <br>
r_1h = sademäärä 1h (mm) <br>
ri_10min = sateen intensiteetti (mm/h) <br>
snow_aws = lumen syvyys (cm)<br>
p_sea = ilmanpaine (hPa) <br>
vis = näkyvyys (m) <br>
n_man = pilvien määrä (1/8) <br>
wawa = vallitseva sää (luokittelu, katso https://www.ilmatieteenlaitos.fi/latauspalvelun-pikaohje)<br>

## Ladataan yllä olevien kokeilujen perusteella alueen säätiedot
ajalta 2016-2021, tammi, helmi ja maaliskuut

In [30]:
# Tiedonkeruun ajoittamiseksi ja palvelimen ylilataamisen välttämiseksi käytetään time
# -kirjaston sleep metodia.
import time

In [31]:
# Funktio tarvittavien osoitteiden rakentamiseen.
def fmi_url_constructor(start_time, end_time, query, parameters):
    """
    Construct url to fetch data from Finnish Meteorological Institute API.
    @param start_time: datetime object.
    @param end_time: datetime object.
    @param query: storedquery id specified by the wfs service specifications.
    @param parameters: url parameters in a dictionary to include.
    
    @return constructed url.
    """
    base_url = "http://opendata.fmi.fi/wfs/fin?"
    url = base_url

    for param in parameters:
        value = parameters[param]
        if isinstance(value, datetime.date):
            value = value.strftime("%Y-%m-%dT%H:%M:%SZ")
        url += param + "=" +str(value)+"&"
        
    return url

In [49]:
# Tallennetaan aikavälien säähavainnot lopulta yhteen kirjastoon.
# Kirjaston avaimina toimivat arvojen otsikot. Tulokset kirjataan päiväkohtaisesti.
weather = {
    "pvä":[],
    "lämpötila":[],
    "tuulen_nopeus": [],
    "ilman_kosteus":[],
    "sademäärä":[],
    "lumen_syvyys":[],
    "näkyvyys":[],
    "pilvien_määrä":[],
    "sää":[]
}

# Lasketaan kunkin vuoden päivien määrä 
years = [2016,2017,2018,2019,2020,2021] #
days_per_year = []
for year in years:
    startdate = datetime.datetime(year, 1, 1, 12, 0)
    enddate = datetime.datetime(year, 4, 1, 12, 0)
    days = (enddate-startdate).days
    days_per_year.append(days)

In [50]:
# Tarkistetaan, että lasku meni oikein.
days_per_year

[91, 90, 90, 90, 91, 90]

In [61]:
# =========================================
# Rakennetaan kyselyfunktio. 
# Kyselyfunktio jakaa haettavat päivät 7 päivän "lohkoihin", koska kyselyjä pystyi tekemään vain 7 päivää kerrallaan.
# Funktio palauttaa python kirjaston, jossa avaimina toimivat kyselystä saadut arvot ja arvoina kyselyn arvot lohkoina.
# =========================================

# TODO: Käännä englannista selitykset vielä suomeksi.
def fetch_weather():
    iterations = 6 # Tehtävien iteraatioiden määrä. Haetaan tietoa kuudelta vuodelta, joten iteraatioita tarvitaan 6.
    
    timespan_in_days = 7 # Yhden patchin koko.

    wait_time = 3 # Kuinka kauan (sekuntteina) odotetaan ennen uuden kyselyn tekemistä? Tämä varmistaa, ettei kyselyjä tehdä palvelimelle liian nopeasti.

    total_requests = 0 # Kuinka monta kyselyä on tehty yhteensä.
    
        
    def get_patch(cur_patch):
        """
        Helper function to fetch data one patch at a time.
        """
        start_time = cur_patch[0] # The first date in the patch.
        end_time = cur_patch[-1] # The last date in the patch.
        
        query = "fmi::observations::weather::timevaluepair"

        parameters = {
            "service": "WFS", # WFS Specific information
            "version": "2.0.0",
            "request": "GetFeature",
            "storedquery_id": query, # Query id. What kind of data are we looking for?
            "starttime": start_time, # datetime
            "endtime": end_time, # datetime
            "place": "Kokemäki", # Place name
            "maxlocations": 1,
            "timestep": 1440, # Minutes in day
            "parameters":"t2m,ws_10min,rh,r_1h,snow_aws,vis,n_man,wawa" # Lämpötila, tuulen nopeus, ilman kosteus, sademäärä, lumen syvyys, näkyvyys, pilvien määrä, vallitseva sää
        }

        url = fmi_url_constructor(start_time, end_time, query, parameters)
        
        response = requests.get(url)
        
        if response.status_code == 200: # Everything OK, response get.
            return response.text
        else: # Else if something unexpected happened.
            print(response.status_code)
            return False 
        
    
    # Start fetching data.
    
    # Collect all the fetched data into this object.
    weather_data = {
        "date":[],
        "t2m":[],
        "ws_10min":[],
        "rh":[],
        "r_1h":[],
        "snow_aws":[],
        "vis":[],
        "n_man":[],
        "wawa":[]
    }
    
    for iteration in range(iterations):
        print(f"Starting iteration {iteration}.")
        cur_year = years[iteration] # Starting from 2016.
        cur_day = 0 # Days that have been fetched
        total_days = days_per_year[iteration] # How many days must be fetched per year.
        patches = [] # Patches of days that will be fetched as one package.
        
        # Create the patches
        for patch in range(0, total_days+1, timespan_in_days):
            startdate = datetime.datetime(cur_year, 1, 1, 0, 0)+datetime.timedelta(days=patch-1)
            enddate = startdate+datetime.timedelta(days=7)         
            dates = [] # Intermediate list to hold the date values.
            dates.append(startdate)
            dates.append(enddate)
            patches.append(dates) # Add the patch to the patches list.
        
        # Fetch and parse the data.
        for patch in patches:
            print(f"Retrieving patch {patch}.")
            response = get_patch(patch)
            if response == False:
                break
            print("Patch OK!")
            total_requests += 1
            data = xmltodict.parse(response)
                
            patch_values = {}
            # Each X corresponds to a set of 
            for x in data["wfs:FeatureCollection"]["wfs:member"]:
                y = x["omso:PointTimeSeriesObservation"]["om:result"]["wml2:MeasurementTimeseries"]
                key = y["@gml:id"].rsplit("-",1)[1] # Split the key string by - starting from the right. Get only the relevant key.
                days = y["wml2:point"] # Every fetched value for the key in the patch.
                values = []
                for day in days:
                    value = day["wml2:MeasurementTVP"]["wml2:value"]
                    values.append(value)
                patch_values[key] = values
                timerange = (patch[1]-patch[0]).days # Get the amount of days between start and end dates.
            patch_values["date"]=[patch[0]+datetime.timedelta(days=x) for x in range(timerange)] # Generate dates for the patch.
            
            # Add the values to the main data object.
            ptch_keys = list(patch_values.keys())
            [weather_data[x].append(patch_values[x]) for x in ptch_keys]
            
            print(f"Patch {patch} done!")
            print(f"Total amount of requests: {total_requests}.")
            time.sleep(wait_time) # Wait for a few seconds before next iteration.
            
    return weather_data

In [106]:
# Suoritetaan yllä oleva funktio ja tallennetaan vastaukset "weather_data" -objektiin.
weather_data = fetch_weather()

Starting iteration 0.
Retrieving patch [datetime.datetime(2015, 12, 31, 0, 0), datetime.datetime(2016, 1, 7, 0, 0)].
Patch OK!
Patch [datetime.datetime(2015, 12, 31, 0, 0), datetime.datetime(2016, 1, 7, 0, 0)] done!
Total amount of requests: 1.
Retrieving patch [datetime.datetime(2016, 1, 7, 0, 0), datetime.datetime(2016, 1, 14, 0, 0)].
Patch OK!
Patch [datetime.datetime(2016, 1, 7, 0, 0), datetime.datetime(2016, 1, 14, 0, 0)] done!
Total amount of requests: 2.
Retrieving patch [datetime.datetime(2016, 1, 14, 0, 0), datetime.datetime(2016, 1, 21, 0, 0)].
Patch OK!
Patch [datetime.datetime(2016, 1, 14, 0, 0), datetime.datetime(2016, 1, 21, 0, 0)] done!
Total amount of requests: 3.
Retrieving patch [datetime.datetime(2016, 1, 21, 0, 0), datetime.datetime(2016, 1, 28, 0, 0)].
Patch OK!
Patch [datetime.datetime(2016, 1, 21, 0, 0), datetime.datetime(2016, 1, 28, 0, 0)] done!
Total amount of requests: 4.
Retrieving patch [datetime.datetime(2016, 1, 28, 0, 0), datetime.datetime(2016, 2, 4, 0,

In [105]:
# Tarkistetaan, miltä kerätty data näyttää.
weather_data

{'date': [[datetime.datetime(2015, 12, 31, 0, 0),
   datetime.datetime(2016, 1, 1, 0, 0),
   datetime.datetime(2016, 1, 2, 0, 0),
   datetime.datetime(2016, 1, 3, 0, 0),
   datetime.datetime(2016, 1, 4, 0, 0),
   datetime.datetime(2016, 1, 5, 0, 0),
   datetime.datetime(2016, 1, 6, 0, 0)],
  [datetime.datetime(2016, 1, 7, 0, 0),
   datetime.datetime(2016, 1, 8, 0, 0),
   datetime.datetime(2016, 1, 9, 0, 0),
   datetime.datetime(2016, 1, 10, 0, 0),
   datetime.datetime(2016, 1, 11, 0, 0),
   datetime.datetime(2016, 1, 12, 0, 0),
   datetime.datetime(2016, 1, 13, 0, 0)],
  [datetime.datetime(2016, 1, 14, 0, 0),
   datetime.datetime(2016, 1, 15, 0, 0),
   datetime.datetime(2016, 1, 16, 0, 0),
   datetime.datetime(2016, 1, 17, 0, 0),
   datetime.datetime(2016, 1, 18, 0, 0),
   datetime.datetime(2016, 1, 19, 0, 0),
   datetime.datetime(2016, 1, 20, 0, 0)],
  [datetime.datetime(2016, 1, 21, 0, 0),
   datetime.datetime(2016, 1, 22, 0, 0),
   datetime.datetime(2016, 1, 23, 0, 0),
   datetime.d

In [107]:
# Datassa on duplikaattiarvoja arvoilla. Poistetaan nämä.

weather_data_c = weather_data.copy() # Varmuuskopio siltä varalta, jos toimenpiteet eivät toimikkaan.

columns = list(weather_data_c.keys())
columns.pop(0)

for column in columns:
    for patch in weather_data_c[column]:
        patch.pop(0) # Poistetaan ensimmäinen arvo jokaisesta aikaisemmin kerätystä lohkosta. Ensimmäinen arvo vastaa aina edellisen lohkon viimeistä arvoa, joten niitä ei tarvita.
        
weather_data_c

{'date': [[datetime.datetime(2015, 12, 31, 0, 0),
   datetime.datetime(2016, 1, 1, 0, 0),
   datetime.datetime(2016, 1, 2, 0, 0),
   datetime.datetime(2016, 1, 3, 0, 0),
   datetime.datetime(2016, 1, 4, 0, 0),
   datetime.datetime(2016, 1, 5, 0, 0),
   datetime.datetime(2016, 1, 6, 0, 0)],
  [datetime.datetime(2016, 1, 7, 0, 0),
   datetime.datetime(2016, 1, 8, 0, 0),
   datetime.datetime(2016, 1, 9, 0, 0),
   datetime.datetime(2016, 1, 10, 0, 0),
   datetime.datetime(2016, 1, 11, 0, 0),
   datetime.datetime(2016, 1, 12, 0, 0),
   datetime.datetime(2016, 1, 13, 0, 0)],
  [datetime.datetime(2016, 1, 14, 0, 0),
   datetime.datetime(2016, 1, 15, 0, 0),
   datetime.datetime(2016, 1, 16, 0, 0),
   datetime.datetime(2016, 1, 17, 0, 0),
   datetime.datetime(2016, 1, 18, 0, 0),
   datetime.datetime(2016, 1, 19, 0, 0),
   datetime.datetime(2016, 1, 20, 0, 0)],
  [datetime.datetime(2016, 1, 21, 0, 0),
   datetime.datetime(2016, 1, 22, 0, 0),
   datetime.datetime(2016, 1, 23, 0, 0),
   datetime.d

In [110]:
# Data on vielä omissa lohkoissaan. Puretaan lohkot ja tallennetaan "weather" -objektiin.
weather["pvä"] = [value for interlist in weather_data_c["date"] for value in interlist]
weather["lämpötila"] = [value for interlist in weather_data_c["t2m"] for value in interlist]
weather["tuulen_nopeus"] = [value for interlist in weather_data_c["ws_10min"] for value in interlist]
weather["ilman_kosteus"] = [value for interlist in weather_data_c["rh"] for value in interlist]
weather["sademäärä"] = [value for interlist in weather_data_c["r_1h"] for value in interlist]
weather["lumen_syvyys"] = [value for interlist in weather_data_c["snow_aws"] for value in interlist]
weather["näkyvyys"] = [value for interlist in weather_data_c["vis"] for value in interlist]
weather["pilvien_määrä"] = [value for interlist in weather_data_c["n_man"] for value in interlist]
weather["sää"] = [value for interlist in weather_data_c["wawa"] for value in interlist]

In [112]:
# Tehdään datasta pandas dataframe.

weather_df = pd.DataFrame(weather)

In [113]:
weather_df

,pvä,lämpötila,tuulen_nopeus,ilman_kosteus,sademäärä,lumen_syvyys,näkyvyys,pilvien_määrä,sää
0,2015-12-31,-5.1,5.5,87.0,0.0,0.0,20401.0,3.0,0.0
1,2016-01-01,-9.5,2.8,89.0,0.0,0.0,48925.0,0.0,0.0
2,2016-01-02,-10.9,1.6,91.0,0.0,0.0,14428.0,8.0,71.0
3,2016-01-03,-9.6,4.2,89.0,0.0,2.0,7509.0,7.0,71.0
4,2016-01-04,-17.0,2.1,84.0,0.0,2.0,45445.0,3.0,24.0
...,...,...,...,...,...,...,...,...,...
555,2021-03-27,5.6,3.6,76.0,0.0,0.0,17954.0,7.0,0.0
556,2021-03-28,3.4,5.0,90.0,0.0,0.0,16036.0,7.0,0.0
557,2021-03-29,6.3,4.0,92.0,0.0,0.0,42534.0,7.0,0.0
558,2021-03-30,4.3,3.1,67.0,0.0,0.0,48655.0,8.0,81.0


In [128]:
# Muutetaan data oikeiksi tiedostotyypeiksi.

weather_df["lämpötila"] = weather_df["lämpötila"].astype("float64")
weather_df["tuulen_nopeus"] = weather_df["tuulen_nopeus"].astype("float64")
weather_df["ilman_kosteus"] = weather_df["ilman_kosteus"].astype("float64")
weather_df["sademäärä"] = weather_df["sademäärä"].astype("float64")
weather_df["lumen_syvyys"] = weather_df["lumen_syvyys"].astype("float64")
weather_df["näkyvyys"] = weather_df["näkyvyys"].astype("float64")
weather_df["pilvien_määrä"] = weather_df["pilvien_määrä"].astype("float64")
weather_df["sää"] = weather_df["sää"].astype("float64")

weather_df

,pvä,lämpötila,tuulen_nopeus,ilman_kosteus,sademäärä,lumen_syvyys,näkyvyys,pilvien_määrä,sää
0,2015-12-31,-5.1,5.5,87.0,0.0,0.0,20401.0,3.0,0.0
1,2016-01-01,-9.5,2.8,89.0,0.0,0.0,48925.0,0.0,0.0
2,2016-01-02,-10.9,1.6,91.0,0.0,0.0,14428.0,8.0,71.0
3,2016-01-03,-9.6,4.2,89.0,0.0,2.0,7509.0,7.0,71.0
4,2016-01-04,-17.0,2.1,84.0,0.0,2.0,45445.0,3.0,24.0
...,...,...,...,...,...,...,...,...,...
555,2021-03-27,5.6,3.6,76.0,0.0,0.0,17954.0,7.0,0.0
556,2021-03-28,3.4,5.0,90.0,0.0,0.0,16036.0,7.0,0.0
557,2021-03-29,6.3,4.0,92.0,0.0,0.0,42534.0,7.0,0.0
558,2021-03-30,4.3,3.1,67.0,0.0,0.0,48655.0,8.0,81.0


In [129]:
# Tarkistetaan, miltä näyttää.
weather_df.describe()

,lämpötila,tuulen_nopeus,ilman_kosteus,sademäärä,lumen_syvyys,näkyvyys,pilvien_määrä,sää
count,560.000000,560.000000,560.000000,559.000000,559.000000,560.000000,560.000000,560.000000
mean,-3.560000,3.115357,88.078571,0.044902,7.513417,27628.117857,5.467857,22.092857
std,6.403158,1.840232,9.385268,0.215372,8.888159,17929.505948,3.335026,31.692766
min,-26.300000,0.000000,47.000000,0.000000,-1.000000,342.000000,0.000000,0.000000
25%,-6.525000,1.800000,84.000000,0.000000,0.000000,9538.000000,1.000000,0.000000
50%,-1.700000,2.900000,90.000000,0.000000,4.000000,27179.500000,7.000000,0.000000
75%,0.900000,4.000000,95.000000,0.000000,13.000000,47523.750000,8.000000,52.000000
max,7.600000,10.700000,100.000000,2.700000,45.000000,50000.000000,9.000000,85.000000


In [133]:
# Tallennetaan säätiedot .csv ja .xlsx -tiedostoon.

weather_df.to_csv("weather_data_2016-2021_kokemaki.csv", index=False)

In [134]:
weather_df["pvä"]

0     2015-12-31
1     2016-01-01
2     2016-01-02
3     2016-01-03
4     2016-01-04
         ...    
555   2021-03-27
556   2021-03-28
557   2021-03-29
558   2021-03-30
559   2021-03-31
Name: pvä, Length: 560, dtype: datetime64[ns]